# Création d'une grille curvilinéaire non-structurée à l'aide du module Pygrid

### Importation des librairies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pygridgen

ModuleNotFoundError: No module named 'pygridgen'

### On crée trois fonctions qui nous permettent de mieux extraire les données provenant des grilles curvilinéaires

In [ ]:
# Il serait possible d'importer ces fonctions, mais pour l'exercice,
# je les laisse ici.

def create_maps(xpos,ypos) :
    """Fonction qui utilise les positions (x,y) de chaque noeuds
    pour créer un vecteur (carte/maps) qui associe chaque noeuds 
    à un nombre (INT). Le vecteur résultant possède la même forme
    que les grilles entrantes."""
    
    nx = xpos.shape[1]
    ny = xpos.shape[0]
    maps = np.zeros(xpos.shape)
    inode = 0
    for j in range(ny) :
        for i in range(nx) :
            maps[j,i] = int(inode)
            inode += 1
    return maps
# -------------------------------


# -------------------------------
def get_mapping(maps, xpos, ypos) :
    """Fonction qui prend la carte (np.array) et les positions
    (x,y) de chaque noeuds et retourne un dictionnaire avec
    comme clef (keys) l'identifiant du noeuds et comme valeur
    (value) un tuple de position (x,y)"""
    
    return {i:(j,k) for i,j,k in zip(maps.flatten(),
                                     grid.x.flatten(),
                                     grid.y.flatten())}
# -------------------------------


# -------------------------------
def extract_shapes(maps) :
    """Fonction qui crée les quadrilatères à l'aide de la carte
    précédente et retourne un np.array contenant toutes les
    connections entre chaque noeuds (shapes)."""
    
    nx = maps.shape[1]
    ny = maps.shape[0]
    shapes = {}
    ishape = 0
    for j in range(ny-1) :
        for i in range(nx-1) :
            shapes[ishape] = [maps[j,i],
                             maps[j,i+1],
                             maps[j+1,i+1],
                             maps[j+1,i]]
            ishape += 1
    return shapes
# -------------------------------

### Création d'une grille avec une forme étrange

In [ ]:
# On crée préalablement le contour de notre grille et on associe
# un courbure (beta) à chacun de nos coins.
x = np.array([0.5, 2.0, 3.5, 2.5, 1.5, 0.5]) 
y = np.array([0.5, 0.5, 1.0, 2.0, 2.0, 0.5])
beta = np.array([1, 1, 0, 1, 1, 0])

# beta = 1 => Coin vers l'intérieur
# beta = 0 => Pas de courbure de grille
# beta = -1 => Coin vers l'extérieur



# Figure : 
fig, data_ax = plt.subplots(figsize=(8.5, 5), ncols=1)
data_ax.grid()
data_ax.fill(x,y,color='black', facecolor='aliceblue')
data_ax.set_title("Illustration d'une forme arbitraire à remplir avec Pygrid.")
for ix,iy,ibeta in zip(x,y,beta) :
    data_ax.annotate(r'$\beta=${}'.format(int(ibeta)),
                     (ix,iy), 
                     color='blue')
data_ax.set_axisbelow(True)

In [ ]:
#  === création de la grille + extraction des données de grille : 
grid = pygridgen.Gridgen(x, y, beta, shape=(10, 10))
maps = create_maps(grid.x,grid.y)
mapping = get_mapping(maps, grid.x, grid.y)
shapes = extract_shapes(maps)

# === Création d'une figure === #
fig, data_ax = plt.subplots(figsize=(8.5, 5), ncols=1)

# Plotting each shapes :
for values in shapes.values() :
    # Getting positions of each nodes for each shapes
    xquad = [mapping[inode][0] for inode in values]
    yquad = [mapping[inode][1] for inode in values]
    # Plotting shapes-frontiers : 
    data_ax.fill(xquad,yquad,
                 color = 'black',
                 linestyle = 'dotted',
                 linewidth = 0.3,
                 facecolor='aliceblue')
    
# Plotting each node numbers :
for ix, iy, inode in zip(grid.x_rho.flatten(),
                         grid.y_rho.flatten(),
                         list(shapes.keys())) :
    data_ax.annotate(int(inode),(ix,iy),
                     ha='center',va='center',c='blue')

# Plotting contours :
data_ax.plot(x,y, c='black', label='Contours', linewidth = 1)


# Figure fine-tunning: 
data_ax.set_axisbelow(True)
data_ax.legend()
data_ax.set_title("Application d'une grille sur une forme arbitraire.")
fig.tight_layout()
#fig.show()

# Application avec Xugrid

In [ ]:
# On importe les librairies : 
import xarray as xr
import xugrid as xu

# On crée une bathymétrie aléatoire pour illustrer le processus : 
bathymetrie = np.sin(grid.x)

In [ ]:
# On crée le Dataset qui représente les noeuds : 
ds = xr.Dataset({'bathymetrie':(['node'],bathymetrie.flatten()),
                'latitude':(['node'],grid.y.flatten()),
                 'longitude':(['node'],grid.x.flatten()),
                 'face_node_connectivity':(['face','nmax_face'],list(shapes.values())),
                 'mesh2d':([''],[])})


ds.face_node_connectivity.attrs = {'cf_role': 'face_node_connectivity',
                                   'long_name': 'Vertex nodes of mesh faces (counterclockwise)',
                                   'start_index': 0}


ds.mesh2d.attrs = {'cf_role': 'mesh_topology',
                   'long_name': 'Topology data of 2D mesh',
                   'topology_dimension': 2,
                   'node_coordinates': 'longitude latitude',
                   'face_node_connectivity': 'face_node_connectivity',
                   'edge_node_connectivity': 'edge_node_connectivity'}
# On montre : 
ds


In [ ]:
uds = xu.UgridDataset(ds)
bathy = uds.bathymetrie

# === création d'une figure === #
bathy.ugrid.plot(cmap='viridis',figsize = (12, 5))
bathy.ugrid.plot.line(color = 'black', linewidth = 0.7, linestyle = 'dashed', label = 'Grillage')
plt.scatter(grid.x_rho,grid.y_rho,marker = 'D', c='black', label='Centroides')
plt.legend()
fig.tight_layout()
#plt.ylabel('Position en y')
#plt.xlabel('Position en x')
#fig.show()